# Initializing notebook

Initializing the notebook and MSTICPy.

In [ ]:
# import some modules needed in this cell
from IPython.display import display, HTML

display(HTML("Checking upgrade to latest msticpy version"))
%pip install --upgrade --quiet msticpy[azuresentinel]

REQ_PYTHON_VER="3.8"
REQ_MSTICPY_VER="1.5.2"

# initialize msticpy
import msticpy
msticpy.init_notebook(namespace=globals());

# Getting Workspace and Authenticating

In [ ]:
ws_config = WorkspaceConfig()

if not ws_config.config_loaded:
    ws_config.prompt_for_ws()
  
qry_prov = QueryProvider(data_environment="AzureSentinel")

qry_prov.connect(ws_config, mp_az_auth=False)
table_index = qry_prov.schema_tables

# Getting target node

Node to use when building shortest path to.

In [ ]:
WIDGET_DEFAULTS = {
    "layout": widgets.Layout(width="95%"),
    "style": {"description_width": "initial"},
}
node_text = widgets.Text(description='Enter the Node to search for:', **WIDGET_DEFAULTS)
display(node_text)

In [ ]:
target_node = node_text.value
print(target_node)

# Identifying shortest path to node

Using ingested graph data, the shortest path to desired node is constructed.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Get nodes and Edges from Custom logs
nodes_df = qry_prov.exec_query('UBHnodes_CL | project Id_d,  Name=strcat("[", toint(Id_d),"] ", replace_string(iff(Properties_azname_s == "",Properties_objectid_g, Properties_azname_s ), ".ONMICROSOFT.COM","")),color = "green"')
edges_df = qry_prov.exec_query('UBHrelationships_CL | where Label_s != "AZContains" | extend StartId = toint(Start_id_s), EndId = toint(End_id_s)')

# Get target node
target_node_df = qry_prov.exec_query("""
UBHnodes_CL | where Properties_azname_s == '{0}' | project Id_d
""".format(target_node))
target_node_id = int(target_node_df.Id_d)

# Build the Graph object
G = nx.Graph()
# Add the edges
G = nx.from_pandas_edgelist(edges_df, 'StartId', 'EndId', 'Label_s', create_using=nx.DiGraph())
node_attr = nodes_df.set_index('Id_d').to_dict('index')
nx.set_node_attributes(G, node_attr)

# Get the shortest path to a node, using the tenant as the target node here, but it could be any node
path = nx.single_target_shortest_path(G, target_node_id)

# Get the 'filtered' edges, and Graph
subG_loops = G.subgraph(path)
subG = nx.DiGraph(subG_loops)

# Remove unwanted edges
length = dict(nx.single_target_shortest_path_length(subG, target_node_id))
backward_edges = [(v,u) for v,u in subG.edges() if length[u] > length[v]]
subG.remove_edges_from(backward_edges)

# Get Edges labels
edge_labels = nx.get_edge_attributes(subG, 'Label_s')
node_labels = nx.get_node_attributes(subG, 'Name')

# Helper functions
def move_labels(pos, x_shift, y_shift):
    return {n:(x + x_shift, y + y_shift) for n,(x,y) in pos.items()}

def get_longest_path(paths):
    return max((len(v), v, k) for k, v in paths.items())[1:]

# Determine "complete" paths
unique_paths = {}
for i,n in enumerate(path):
    unique_paths[n] = 1
    for value in [path_value for path_value in path.values() if path_value != path[n]]:
        if (set(path[n]).issubset(set(value))):
            unique_paths.update({n:0})
            break

# define column positions
pos = {n: (len(get_longest_path(path)[0]) - len(path[n]), 0) for i, n in enumerate(path)}

# define row positions
y = 0
for i, j in enumerate(path):
    if unique_paths[j] == 1:
        y += 1
        pos.update({n: (pos[n][0], pos[n][1] + y) for n in path[j]})

# update target node position
pos.update({target_node_id: (pos[target_node_id][0], y/2 + 0.5)})

# shift label positions above nodes
pos_labels = move_labels(pos, 0, 0.1)

plt.figure(figsize=(20, 10))
plt.title("Shortest path to node")
nx.draw_networkx(subG, pos, with_labels=False)
nx.draw_networkx_edge_labels(subG, pos, edge_labels=edge_labels)
nx.draw_networkx_labels(subG, pos_labels, labels = node_labels) 
plt.show()

# Getting alerts and events

Querying for alert and event information to enrich graph. 

In [ ]:
# Get nodes and Edges from Incidents and Events

time_frame = "4d"

# Get related incidents
def get_related_incidents_by_node_id(aNodeIds):
    queryToExecute ="""
    SecurityAlert
    | where TimeGenerated > ago({0})
    | mv-expand parse_json(Entities)
    | extend EntityName = toupper(tostring(Entities["DisplayName"]))
    | where EntityName != ""
    | join kind=leftouter UBHnodes_CL on $left.EntityName == $right.Properties_azname_s
    | project SystemAlertId, StartId = toint(substring(tostring(replace_regex(SystemAlertId, @'[a-zA-Z-]',"")),0,5)), EndId = toint(Id_d), Label_s = "Incident", Start_labels_s =AlertSeverity, End_labels_s=Labels_s
    | where EndId in ({1})
    """.format(time_frame, ','.join([str(i) for i in aNodeIds]))

    incident_edges_df = qry_prov.exec_query(queryToExecute)

    return incident_edges_df

# Get related events. For prototyping, this includes only "Reset password" events, hinted from the previous related path.
# For next iterations, this would include all related events, depending on the path of interest, broken down to specific event types for each hop
def get_related_events_by_node_id(aNodeIds):
    queryToExecute ="""
    AuditLogs
    | where TimeGenerated > ago({0})
    | where Category =~ "UserManagement"
    | where OperationName == "Reset user password"
    | mv-expand TargetResources
    | extend Source = tostring(parse_json(parse_json(InitiatedBy).user).userPrincipalName)
    | extend Target = toupper(parse_json(TargetResources).userPrincipalName)
    | extend StartId = 2000 + hash(Id, 2000)
    | join kind=leftouter UBHnodes_CL on $left.Target == $right.Properties_azname_s
    | project Id, StartId, EndId = toint(Id_d), Label_s = "Event", Start_labels_s = OperationName, End_labels_s=Labels_s
    | where EndId in ({1})
    """.format(time_frame, ','.join([str(i) for i in aNodeIds]))
  
    event_edges_df = qry_prov.exec_query(queryToExecute)

    return event_edges_df

# Get incident node info
incident_nodes_df = qry_prov.exec_query("""
SecurityAlert
| where TimeGenerated > ago({0})
| project TimeGenerated, Id_d = toint(substring(replace_regex(SystemAlertId, @'[a-zA-Z-]',""),0,5)), Properties_azname_s="Incident", Name = substring(DisplayName,0,50), color="red"
""".format(time_frame))

# Get event node info. For prototyping, this includes only "Reset password" events, hinted from the previous related path.
# For next iterations, this would include all related events, depending on the path of interest, broken down to specific event types for each hop
event_nodes_df = qry_prov.exec_query("""
AuditLogs
| where TimeGenerated > ago({0})
| extend Source = tostring(parse_json(parse_json(InitiatedBy).user).userPrincipalName)
| extend Id_d = 2000 + hash(Id, 2000)
| project TimeGenerated, Id_d, Properties_azname_s="Event", Name = strcat(OperationName, " by ", Source), color = "yellow"
""".format(time_frame))

# Adding incidents to graph

For the identified path, related incidents are retrieved and added to the graph. 

In [ ]:
# set the Graph with incidents added
incidentsG = subG

# Get related Incidents
incidents_edges_df = get_related_incidents_by_node_id(list(subG))
if not incidents_edges_df.empty:
    # create a new Graph object
    incidentsG = nx.Graph()
    # populate with Incidents edges from panda data frame
    incidentsG = nx.from_pandas_edgelist(incidents_edges_df, 'StartId', 'EndId', 'Label_s', create_using=nx.DiGraph())
    #print(incident_nodes_df)
    incidents_node_attr = incident_nodes_df.set_index('Id_d').to_dict('index')    
    nx.set_node_attributes(incidentsG, incidents_node_attr)
    # compose the two graphs
    incidentsG = nx.compose(subG, incidentsG)

# Get Edges labels
edge_labels = nx.get_edge_attributes(incidentsG, 'Label_s')
node_labels = nx.get_node_attributes(incidentsG, 'Name')
node_colors = list(nx.get_node_attributes(incidentsG, 'color').values())

# Set incident node positions
pos_new = {r.StartId: (pos[r.EndId][0], pos[r.EndId][1] - 0.5) for i, r in incidents_edges_df.iterrows()}
pos.update(pos_new)
# Set incident label positions
pos_labels.update(move_labels(pos_new, 0, -0.1))

plotTitle = "Shortest path to node with incidents"
plt.figure(figsize=(20, 10))
plt.title(plotTitle)
nx.draw_networkx(incidentsG, pos, with_labels=False, node_color=node_colors)
nx.draw_networkx_edge_labels(incidentsG, pos, edge_labels=edge_labels)
nx.draw_networkx_labels(incidentsG, pos_labels, labels = node_labels)
plt.show()

# Adding events to graph

For the identified path, related events are retrieved and added to the graph. 

In [ ]:
# set the Graph with events added
eventsG = incidentsG

# Get related Incidents
event_edges_df = get_related_events_by_node_id(list(incidentsG))
#print (event_edges_df)
if not event_edges_df.empty:
    # create a new Graph object
    eventsG = nx.Graph()
    # populate with event edges from panda data frame
    eventsG = nx.from_pandas_edgelist(event_edges_df, 'StartId', 'EndId', 'Label_s', create_using=nx.DiGraph())
    #print(event_nodes_df)
    events_node_attr = event_nodes_df.set_index('Id_d').to_dict('index')
    #print(events_node_attr)
    nx.set_node_attributes(eventsG, events_node_attr)
    # compose the two graphs
    eventsG = nx.compose(incidentsG, eventsG)

# Get Edges labels
edge_labels = nx.get_edge_attributes(eventsG, 'Label_s')
node_labels = nx.get_node_attributes(eventsG, 'Name')
node_colors = list(nx.get_node_attributes(eventsG, 'color').values())

# Set event node positions
pos_new = {r.StartId: (pos[r.EndId][0] + 0.25, pos[r.EndId][1] - 1) for i, r in event_edges_df.iterrows()}
pos.update(pos_new)
# Move event label positions
pos_labels.update(move_labels(pos_new, 0, -0.1))

plotTitle = "Shortest path to node with events"
plt.figure(figsize=(20, 10))
plt.title(plotTitle)
nx.draw_networkx(eventsG, pos, with_labels=False, node_color=node_colors)
nx.draw_networkx_edge_labels(eventsG, pos, edge_labels=edge_labels)
nx.draw_networkx_labels(eventsG, pos_labels, labels = node_labels)

plt.show()